In [1]:
import numpy as np
import pandas as pd

import cv2
import os

In [2]:
base_dir = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/SpatialRegistration/io/'
output_dir = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Metashape/TalChhapar/images/'

In [3]:
df = pd.read_csv(base_dir + '20230310_SE_Lek1_P1D1_DJI_0190_YOLO_Anchored.csv')
df['best_anchor_frame'] = df['best_anchor_frame'].astype(int)
df.head()

,frame,x,y,idx,best_anchor_frame,transformed_x,transformed_y
0,0,3231.399,58.680,43424,0,3231.399,58.680
1,0,1596.498,174.696,43425,0,1596.498,174.696
2,0,3602.919,276.867,43426,0,3602.919,276.867
3,0,2578.536,318.834,43427,0,2578.536,318.834
4,0,1220.571,437.763,43428,0,1220.571,437.763


In [4]:
# Step 1: Extract unique frame numbers from the 'anchor_frames' column
unique_frames = df['best_anchor_frame'].unique()

# Define the video path
video_path = base_dir + '20230310_SE_Lek1_P1D1_DJI_0190.MP4'  # Specify the correct video path
video_name = os.path.splitext(os.path.basename(video_path))[0]  # Extract video name without extension

# Step 2: Create a directory with the video name to store extracted frames
output_folder = output_dir + f'{video_name}'
os.makedirs(output_folder, exist_ok=True)

# Step 3: Use OpenCV to open the video and extract frames
cap = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open the video.")
else:
    print(f"Total frames in video: {total_frames}")
    
    # Step 4: Iterate through unique frames, read and save them
    for frame_no in unique_frames:
        # Ensure frame number is within the video frame range
        if frame_no >= 0 and frame_no < total_frames:
            # Set the video frame to the specific frame number
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
            
            # Read the frame
            ret, frame = cap.read()
            
            if ret:
                # Create the filename for the frame
                frame_filename = os.path.join(output_folder, f"{video_name}_frame{frame_no}.jpg")
                
                # Save the frame as an image
                cv2.imwrite(frame_filename, frame)
                print(f"Saved frame {frame_no}")
            else:
                print(f"Error: Could not read frame {frame_no}")
        else:
            print(f"Frame number {frame_no} is out of bounds")

# Release the video capture object
cap.release()


Total frames in video: 5805
Saved frame 0
Saved frame 405
Saved frame 546
Saved frame 594
Saved frame 605
Saved frame 613
Saved frame 619
Saved frame 630
Saved frame 641
Saved frame 647
Saved frame 654
Saved frame 665
Saved frame 675
Saved frame 687
Saved frame 694
Saved frame 700
Saved frame 710
Saved frame 721
Saved frame 734
Saved frame 879
Saved frame 768
Saved frame 819
Saved frame 835
Saved frame 887
Saved frame 870
Saved frame 912
Saved frame 926
Saved frame 1166
Saved frame 1108
Saved frame 1164
Saved frame 2042
Saved frame 3744
Saved frame 1160
Saved frame 1677
Saved frame 1168
Saved frame 1171
Saved frame 1176
Saved frame 1244
Saved frame 1307
Saved frame 3210
Saved frame 1374
Saved frame 1357
Saved frame 1475
Saved frame 1378
Saved frame 1812
Saved frame 5256
Saved frame 1803
Saved frame 1410
Saved frame 1710
Saved frame 3943
Saved frame 1556
Saved frame 1848
Saved frame 3926
Saved frame 2224
Saved frame 4512
